In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
PWD = os.getenv('PWD')
db_name = PWD+'\\database'+'\\test.db'
df = pd.read_sql(
    '''SELECT * FROM minute_scale 
    WHERE date_created > "2021-12-30 00:00:00.000000"
    
    ''',
    'sqlite:///' + db_name, 
    index_col='id',
)
print(len(df))
df.head(3)
# df

2019


,date_created,open_time,open_,high_,low_,close_,volume_,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore_
id,,,,,,,,,,,,,
1187489,2021-12-30 00:01:00.000000,1640811660000,0.09371000,0.09372000,0.09364000,0.09365000,17377.40000000,1640811719999,1628.14305500,10,4098.60000000,383.83243200,0
1187490,2021-12-30 00:02:00.000000,1640811720000,0.09361000,0.09361000,0.09336000,0.09339000,48914.10000000,1640811779999,4573.68382400,31,19135.60000000,1788.37335600,0
1187491,2021-12-30 00:03:00.000000,1640811780000,0.09337000,0.09337000,0.09311000,0.09312000,54421.70000000,1640811839999,5073.82000600,41,23623.30000000,2202.53614500,0


In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
PWD = os.getenv('PWD')
db_name = PWD+'\\database'+'\\test.db'
import sys
sys.path.insert(1, PWD+'\\modules')
from alg_modules.alg_handler import AlgHandler
from collections import deque
import logging
DEBUG = __debug__ 
LOG_FILE_NAME = 'log_file_name.log'
format = '%(asctime)s [%(levelname)s]: %(message)s'
logger = logging.basicConfig(
    filename=LOG_FILE_NAME if not DEBUG else None, 
    format=format,
    encoding='utf-8', 
    level=logging.INFO, 
)
if not DEBUG:
    logging.getLogger(logger).addHandler(logging.StreamHandler())


In [19]:
from paper_trade import PaperTrader
df = pd.read_sql(
    'SELECT * FROM minute_scale WHERE date_created > "2021-12-30 00:00:00.000000"',
    'sqlite:///' + db_name, 
    index_col='id',
)
window = deque(maxlen=1000)
p_trdr = PaperTrader(
    main_currency_label='RVN',
    secondary_currency_label='USD',
    main_currency_amount=10,
    # secondary_currency_amount=0
    fee=0.1,
)
# init alg
alg = AlgHandler(df=pd.DataFrame([]))
for i, row in df.iterrows():
    window.append(dict(row))
    # logging.debug(window[-1])
    df = pd.DataFrame(window)

        # logging.debug(df['open_'])
    alg.update_data(df)
    alg.calculate(val_col='open_', time_col='open_time',)
    do_trade, trade_type = alg.evaluate()
    if do_trade:
        logging.info(f"[TRADE] {row['date_created']} {row['open_']}  {trade_type}")
        trade_type = "SELL" if trade_type=='raise' else "BUY"
        p_trdr.trade(
            amount=10 if trade_type == "SELL" else 1,
            trade_type=trade_type, # buy main or sell main
            sell_price=float(row['open_']),
            buy_price=float(row['open_']),
        )
    # find close value of candle that have cross
    

2022-01-24 15:27:19,752 [INFO]: [TRADE] 2021-12-30 02:22:00.000000 0.09125000  fall
2022-01-24 15:27:19,753 [INFO]: RVN: 20.95
2022-01-24 15:27:19,753 [INFO]: USD: -1
2022-01-24 15:27:19,754 [INFO]: trade_type='BUY'
2022-01-24 15:27:19,755 [INFO]: buy_price=0.09125
2022-01-24 15:27:20,009 [INFO]: [TRADE] 2021-12-30 03:49:00.000000 0.09225000  raise
2022-01-24 15:27:20,010 [INFO]: RVN: 10.95
2022-01-24 15:27:20,011 [INFO]: USD: -0.08
2022-01-24 15:27:20,011 [INFO]: trade_type='SELL'
2022-01-24 15:27:20,012 [INFO]: sell_price=0.09225
2022-01-24 15:27:20,090 [INFO]: [TRADE] 2021-12-30 04:09:00.000000 0.09114000  fall
2022-01-24 15:27:20,091 [INFO]: RVN: 21.91
2022-01-24 15:27:20,092 [INFO]: USD: -1.08
2022-01-24 15:27:20,092 [INFO]: trade_type='BUY'
2022-01-24 15:27:20,093 [INFO]: buy_price=0.09114
2022-01-24 15:27:20,247 [INFO]: [TRADE] 2021-12-30 04:58:00.000000 0.09181000  raise
2022-01-24 15:27:20,248 [INFO]: RVN: 11.91
2022-01-24 15:27:20,248 [INFO]: USD: -0.16
2022-01-24 15:27:20,24

In [20]:
p_trdr.convert_to_main(price=0.09488)

2022-01-24 15:27:47,497 [INFO]: RVN: 11.3
2022-01-24 15:27:47,499 [INFO]: USD: 0.0
2022-01-24 15:27:47,500 [INFO]: trade_type='BUY'
2022-01-24 15:27:47,500 [INFO]: buy_price=0.09488
2022-01-24 15:27:47,501 [INFO]: RVN: 11.3
2022-01-24 15:27:47,501 [INFO]: USD: 0.0
2022-01-24 15:27:47,502 [INFO]: self.fee=0.1


In [16]:
p_trdr.get_info()

2022-01-24 15:26:49,020 [INFO]: RVN: 0.0
2022-01-24 15:26:49,021 [INFO]: USD: 0.78
2022-01-24 15:26:49,022 [INFO]: self.fee=0.1


- init RVN 10
- raise == 11.3 RVN +13%
- fall RVN: 8.24 


In [7]:
# plot DataFrame
df = pd.read_sql(
    'SELECT * FROM minute_scale WHERE date_created > "2021-12-30 00:00:00.000000"',
    'sqlite:///' + db_name, 
    index_col='id',
)
from plot_modules.candle_plot import CandlePlot
CandlePlot.from_df(
    df=df,
    open_col='open_',
    close_col='close_',
    low_col='low_',
    high_col='high_',
    date_col='date_created',
    GO_WIDTH=1300
)